# Axisymmetric rotor

This notebook demonstrates the process of running a Workbench service on a local machine to solve both 2D general axisymmetric rotor and 3D rotor models using PyMechanical.
It includes steps for uploading project files, executing scripts, downloading results, and displaying output images.

In [1]:
import os
import pathlib

In [2]:
from ansys.workbench.core import launch_workbench
from ansys.mechanical.core import connect_to_mechanical

Launch the Workbench service on the local machine using specific options.
Define the working directory and subdirectories for assets, scripts, and geometry databases (agdb).
The `launch_workbench` function starts a Workbench session with the specified directory.

In [3]:
workdir = pathlib.Path("__file__").parent

In [4]:
assets = workdir / "assets"
scripts = workdir / "scripts"

In [5]:
wb = launch_workbench(client_workdir=str(workdir.absolute()), use_insecure_connection=True)
current_directory = os.getcwd()

Using insecure connection is not recommended. Please see the documentation for your installed product for additional information.


C:\Users\ansys\actions-runner\_work\pyworkbench-examples\pyworkbench-examples\.venv\Lib\site-packages\ansys\tools\common\cyberchannel.py:187: UserWarning: Starting gRPC client without TLS on localhost:51287. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


Upload the project files to the server using the `upload_file_from_example_repo` method.
The files uploaded are `axisymmetric_model.agdb`, `rotor_3d_model.agdb`.

In [6]:
wb.upload_file_from_example_repo("axisymmetric-rotor/agdb/axisymmetric_model.agdb")
wb.upload_file_from_example_repo("axisymmetric-rotor/agdb/rotor_3d_model.agdb")

Uploading axisymmetric_model.agdb:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Uploading axisymmetric_model.agdb: 100%|██████████| 2.52M/2.52M [00:00<00:00, 69.6MB/s]

Uploading rotor_3d_model.agdb:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Uploading rotor_3d_model.agdb: 100%|██████████| 2.52M/2.52M [00:00<00:00, 69.6MB/s]

Execute a Workbench script (`project.wbjn`) to define the project and load the geometry.
The log file is set to `wb_log_file.log` and the name of the system created is stored in `sys_name` and printed.

In [7]:
log_file = 'wblog.txt'
wb.set_log_file(log_file)
sys_name = wb.run_script_file(str(assets / "project.wbjn"), log_level='info')
print(sys_name)

['SYS', 'SYS 4']


Start a PyMechanical server for the system and create a PyMechanical client session to solve the 2D general axisymmetric rotor model.
The project directory is printed to verify the connection.

In [8]:
server_port = wb.start_mechanical_server(system_name=sys_name[1])

In [9]:
mechanical = connect_to_mechanical(ip='localhost', port=server_port)

In [10]:
print(mechanical.project_directory)

C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\


Read and execute the script `axisymmetric_rotor.py` via the PyMechanical client to mesh and solve the 2D general axisymmetric rotor model.
The output of the script is printed.

In [11]:
with open(scripts / "axisymmetric_rotor.py") as sf:
    mech_script = sf.read()
mech_output = mechanical.run_python_script(mech_script)
print(mech_output)

{"Total Deformation": "0.00079262295050867642 [m]", "Total Deformation 2": "0.00093934782100576889 [m]"}


Specify the Mechanical directory for the Modal Campbell Analysis and fetch the working directory path.
Download the solver output file (`solve.out`) from the server to the client's current working directory and print its contents.

In [12]:
mechanical.run_python_script(f"solve_dir=ExtAPI.DataModel.AnalysisList[2].WorkingDir")
result_solve_dir_server = mechanical.run_python_script(f"solve_dir")
print(f"All solver files are stored on the server at: {result_solve_dir_server}")

All solver files are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-6\MECH\


In [13]:
solve_out_path = os.path.join(result_solve_dir_server, "solve.out")

In [14]:
def write_file_contents_to_console(path):
    """Write file contents to console."""
    with open(path, "rt") as file:
        for line in file:
            print(line, end="")

In [15]:
current_directory = os.getcwd()
if os.path.exists(solve_out_path):
    mechanical.download(solve_out_path, target_dir=current_directory)
    solve_out_local_path = os.path.join(current_directory, "solve.out")
    write_file_contents_to_console(solve_out_local_path)
    os.remove(solve_out_local_path)
else:
    print(f"error: solver output file {solve_out_path} was not generated.")


 Ansys Mechanical Enterprise                       


 *------------------------------------------------------------------*
 |                                                                  |
 |   W E L C O M E   T O   T H E   A N S Y S (R)  P R O G R A M     |
 |                                                                  |
 *------------------------------------------------------------------*




 ***************************************************************
 *         ANSYS MAPDL 2025 R2          LEGAL NOTICES          *
 ***************************************************************
 *                                                             *
 * Copyright 1971-2025 Ansys, Inc.  All rights reserved.       *
 * Unauthorized use, distribution or duplication is            *
 * prohibited.                                                 *
 *                                                             *
 * Ansys is a registered trademark of Ansys, Inc. or its       *
 * sub

 PARAMETER _WB_PROJECTSCRATCH_DIR(1) = C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\_ProjectScratch\Scr2D97\

 SET PARAMETER DIMENSIONS ON  _WB_SOLVERFILES_DIR
  TYPE=STRI  DIMENSIONS=      248        1        1

 PARAMETER _WB_SOLVERFILES_DIR(1) = C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-6\MECH\

 SET PARAMETER DIMENSIONS ON  _WB_USERFILES_DIR
  TYPE=STRI  DIMENSIONS=      248        1        1

 PARAMETER _WB_USERFILES_DIR(1) = C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\user_files\
 --- Data in consistent MKS units. See Solving Units in the help system for more

 MKS UNITS SPECIFIED FOR INTERNAL    
  LENGTH        (l)  = METER (M)
  MASS          (M)  = KILOGRAM (KG)
  TIME          (t)  = SECOND (SEC)
  TEMPERATURE   (T)  = CELSIUS (C)
  TOFFSET            = 273.0
  CHARGE        (Q)  = COULOMB
  FORCE         (f)  = NEWTON (N) (KG-M/SEC2)
  HEAT               = JOULE (N-M)

  PRESSURE           = PASCAL (NEWTON/M**2)
  ENERGY   

Specify the Mechanical directory path for the Modal Campbell Analysis and fetch the image directory path.
Download an image file (`tot_deform_2D.png`) from the server to the client's current working directory and display it using `matplotlib`.

In [16]:
from matplotlib import image as mpimg
from matplotlib import pyplot as plt

In [17]:
mechanical.run_python_script(f"image_dir=ExtAPI.DataModel.AnalysisList[2].WorkingDir")
result_image_dir_server = mechanical.run_python_script(f"image_dir")
print(f"Images are stored on the server at: {result_image_dir_server}")

Images are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-6\MECH\


In [18]:
def get_image_path(image_name):
    return os.path.join(result_image_dir_server, image_name)

In [19]:
def display_image(path):
    print(f"Printing {path} using matplotlib")
    image1 = mpimg.imread(path)
    plt.figure(figsize=(15, 15))
    plt.axis("off")
    plt.imshow(image1)
    plt.show()

In [20]:
image_name = "tot_deform_2D.png"
image_path_server = get_image_path(image_name)

In [21]:
# disable image export or it will fail on github build agent due to graphics context
if False and image_path_server != "":
    local_file_path_list = mechanical.download(
        image_path_server, target_dir=current_directory
    )
    image_local_path = local_file_path_list[0]
    print(f"Local image path : {image_local_path}")

    display_image(image_local_path)

Specify the Mechanical directory for the Unbalance Response Analysis and fetch the working directory path.
Download the solver output file (`solve.out`) from the server to the client's current working directory and print its contents.

In [22]:
mechanical.run_python_script(f"solve_dir=ExtAPI.DataModel.AnalysisList[3].WorkingDir")
result_solve_dir_server = mechanical.run_python_script(f"solve_dir")
print(f"All solver files are stored on the server at: {result_solve_dir_server}")

All solver files are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-7\MECH\


In [23]:
solve_out_path = os.path.join(result_solve_dir_server, "solve.out")

In [24]:
def write_file_contents_to_console(path):
    """Write file contents to console."""
    with open(path, "rt") as file:
        for line in file:
            print(line, end="")

In [25]:
if os.path.exists(solve_out_path):
    mechanical.download(solve_out_path, target_dir=current_directory)
    solve_out_local_path = os.path.join(current_directory, "solve.out")
    write_file_contents_to_console(solve_out_local_path)
    os.remove(solve_out_local_path)
else:
    print(f"error: solver output file {solve_out_path} was not generated.")


 Ansys Mechanical Enterprise                       


 *------------------------------------------------------------------*
 |                                                                  |
 |   W E L C O M E   T O   T H E   A N S Y S (R)  P R O G R A M     |
 |                                                                  |
 *------------------------------------------------------------------*




 ***************************************************************
 *         ANSYS MAPDL 2025 R2          LEGAL NOTICES          *
 ***************************************************************
 *                                                             *
 * Copyright 1971-2025 Ansys, Inc.  All rights reserved.       *
 * Unauthorized use, distribution or duplication is            *
 * prohibited.                                                 *
 *                                                             *
 * Ansys is a registered trademark of Ansys, Inc. or its       *
 * sub

You can save, archive and download the project using `download_project_archive()` method.

In [26]:
#wb.download_project_archive("test_name", show_progress=True)

Start a PyMechanical server for the 3D rotor model system and create a PyMechanical client session.
The project directory is printed to verify the connection.

In [27]:
server_port = wb.start_mechanical_server(system_name=sys_name[0])

In [28]:
mechanical = connect_to_mechanical(ip='localhost', port=server_port)

In [29]:
print(mechanical.project_directory)

C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\


Read and execute the script `rotor_3d.py` via the PyMechanical client to mesh and solve the 3D rotor model.
The output of the script is printed.

In [30]:
with open(scripts / "rotor_3d.py") as sf:
    mech_script = sf.read()
mech_output = mechanical.run_python_script(mech_script)
print(mech_output)

{"Total Deformation": "0 [m]", "Total Deformation 2": "0 [m]"}


Specify the Mechanical directory for the Modal Campbell Analysis and fetch the working directory path.
Download the solver output file (`solve.out`) from the server to the client's current working directory and print its contents.

In [31]:
mechanical.run_python_script(f"solve_dir=ExtAPI.DataModel.AnalysisList[2].WorkingDir")
result_solve_dir_server = mechanical.run_python_script(f"solve_dir")
print(f"All solver files are stored on the server at: {result_solve_dir_server}")

All solver files are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-2\MECH\


In [32]:
solve_out_path = os.path.join(result_solve_dir_server, "solve.out")

In [33]:
def write_file_contents_to_console(path):
    """Write file contents to console."""
    with open(path, "rt") as file:
        for line in file:
            print(line, end="")

In [34]:
if os.path.exists(solve_out_path):
    mechanical.download(solve_out_path, target_dir=current_directory)
    solve_out_local_path = os.path.join(current_directory, "solve.out")
    write_file_contents_to_console(solve_out_local_path)
    os.remove(solve_out_local_path)
else:
    print(f"error: solver output file {solve_out_path} was not generated.")


 Ansys Mechanical Enterprise                       


 *------------------------------------------------------------------*
 |                                                                  |
 |   W E L C O M E   T O   T H E   A N S Y S (R)  P R O G R A M     |
 |                                                                  |
 *------------------------------------------------------------------*




 ***************************************************************
 *         ANSYS MAPDL 2025 R2          LEGAL NOTICES          *
 ***************************************************************
 *                                                             *
 * Copyright 1971-2025 Ansys, Inc.  All rights reserved.       *
 * Unauthorized use, distribution or duplication is            *
 * prohibited.                                                 *
 *                                                             *
 * Ansys is a registered trademark of Ansys, Inc. or its       *
 * sub

 PARAMETER _WB_PROJECTSCRATCH_DIR(1) = C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\_ProjectScratch\ScrE76D\

 SET PARAMETER DIMENSIONS ON  _WB_SOLVERFILES_DIR
  TYPE=STRI  DIMENSIONS=      248        1        1

 PARAMETER _WB_SOLVERFILES_DIR(1) = C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-2\MECH\

 SET PARAMETER DIMENSIONS ON  _WB_USERFILES_DIR
  TYPE=STRI  DIMENSIONS=      248        1        1

 PARAMETER _WB_USERFILES_DIR(1) = C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\user_files\
 --- Data in consistent MKS units. See Solving Units in the help system for more

 MKS UNITS SPECIFIED FOR INTERNAL    
  LENGTH        (l)  = METER (M)
  MASS          (M)  = KILOGRAM (KG)
  TIME          (t)  = SECOND (SEC)
  TEMPERATURE   (T)  = CELSIUS (C)
  TOFFSET            = 273.0
  CHARGE        (Q)  = COULOMB
  FORCE         (f)  = NEWTON (N) (KG-M/SEC2)
  HEAT               = JOULE (N-M)

  PRESSURE           = PASCAL (NEWTON/M**2)
  ENERGY   

Specify the Mechanical directory path for the Modal Campbell Analysis and fetch the image directory path.
Download an image file (`tot_deform_3D.png`) from the server to the client's current working directory and display it using `matplotlib`.

In [35]:
from matplotlib import image as mpimg
from matplotlib import pyplot as plt

In [36]:
mechanical.run_python_script(f"image_dir=ExtAPI.DataModel.AnalysisList[2].WorkingDir")
result_image_dir_server = mechanical.run_python_script(f"image_dir")
print(f"Images are stored on the server at: {result_image_dir_server}")

Images are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-2\MECH\


In [37]:
def get_image_path(image_name):
    return os.path.join(result_image_dir_server, image_name)

In [38]:
def display_image(path):
    print(f"Printing {path} using matplotlib")
    image1 = mpimg.imread(path)
    plt.figure(figsize=(15, 15))
    plt.axis("off")
    plt.imshow(image1)
    plt.show()

In [39]:
image_name = "tot_deform_3D.png"
image_path_server = get_image_path(image_name)

In [40]:
# disable image export or it will fail on github build agent due to graphics context
if False and image_path_server != "":
    local_file_path_list = mechanical.download(
        image_path_server, target_dir=current_directory
    )
    image_local_path = local_file_path_list[0]
    print(f"Local image path : {image_local_path}")

    display_image(image_local_path)

Specify the Mechanical directory for the Unbalance Response Analysis and fetch the working directory path.
Download the solver output file (`solve.out`) from the server to the client's current working directory and print its contents.

In [41]:
mechanical.run_python_script(f"solve_dir=ExtAPI.DataModel.AnalysisList[3].WorkingDir")
result_solve_dir_server = mechanical.run_python_script(f"solve_dir")
print(f"All solver files are stored on the server at: {result_solve_dir_server}")

All solver files are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-3\MECH\


In [42]:
solve_out_path = os.path.join(result_solve_dir_server, "solve.out")

In [43]:
def write_file_contents_to_console(path):
    """Write file contents to console."""
    with open(path, "rt") as file:
        for line in file:
            print(line, end="")

In [44]:
if os.path.exists(solve_out_path):
    mechanical.download(solve_out_path, target_dir=current_directory)
    solve_out_local_path = os.path.join(current_directory, "solve.out")
    write_file_contents_to_console(solve_out_local_path)
    os.remove(solve_out_local_path)
else:
    print(f"error: solver output file {solve_out_path} was not generated.")

error: solver output file C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-3\MECH\solve.out was not generated.


Download all the files from the server to the current working directory for the 3D rotor model.
Verify the source path for the directory and copy all files from the server to the client.

In [45]:
import shutil
import glob

In [46]:
target_dir2 = current_directory
print(f"Files to be copied from server path at: {target_dir2}")
print(f"All the solver files are stored on the server at: {result_solve_dir_server}")

Files to be copied from server path at: C:\Users\ansys\actions-runner\_work\pyworkbench-examples\pyworkbench-examples\pyworkbench-examples\doc\source\examples\axisymmetric-rotor
All the solver files are stored on the server at: C:\Users\ansys\AppData\Local\Temp\WB_ansys_7656_2\wbnew_files\dp0\SYS-3\MECH\


In [47]:
source_dir = result_solve_dir_server
destination_dir = target_dir2

In [48]:
for file in glob.glob(source_dir + '/*'):
    shutil.copy(file, destination_dir)

Finally, call the `exit` method on both the PyMechanical and Workbench clients to gracefully shut down the services.

In [49]:
mechanical.exit()
wb.exit()